In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta

In [2]:
# Read the data
# MAke sure that this code file is in the same folder as your data file.
df = pd.read_csv('model_estimation_data.dat', sep='\t')
database = db.Database('model_estimation_data', df)

In [3]:
# The following statement allows you to use the names of the variable as Python variable.
globals().update(database.variables)

In [4]:
# Parameters to be estimated
ASC_Bus    = Beta('ASC_Bus',0,None,None,0)
ASC_Bike   = Beta('ASC_Bike',0,None,None,1)
ASC_Car    = Beta('ASC_Car',0,None,None,0)

# Note that in above case, the parameter ASC_Bike is not estimated (as specified by the 1 in the fifth argument on 
# the corresponding line), and its value is fixed to 0. Read: https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf

# Similarly now, declare parameters for travel time, "Beta_TT" and travel cost, "Beta_TC" below:
Beta_TT    = Beta('Beta_TT', 0, None, None, 0)
Beta_Cost  = Beta('Beta_Cost', 0, None, None, 0)

In [5]:
# Definition of new variables
tot_cost_car = car_Cost + car_PCost

# Similarly now, define new variables for total time for car and total time for bus
tot_time_car = car_TT + car_PT
tot_time_bus = bus_IVT + bus_TA + bus_WT

In [6]:
# Definition of the utility functions (provided for V1-Bus, define yourself for Bike and Car)
V1    =  ASC_Bus + Beta_TT * tot_time_bus  + Beta_Cost * bus_Cost
V2    =  ASC_Bike + Beta_TT * BS_TT + Beta_Cost * BS_Cost
V3    =  ASC_Car + Beta_TT * tot_time_car + Beta_Cost * tot_cost_car

# Associate utility functions with the numbering of alternatives
V = {1: V1, 2: V2, 3: V3}

In [7]:
# Associate the availability conditions with the alternatives
Bus_AV = 1
Bike_AV = 1
Car_AV = 1

av = {1: Bus_AV, 2: Bike_AV, 3: Car_AV}

In [8]:
# Definition of the model. This is the contribution of each observation to the log likelihood function.
logprob = models.loglogit(V, av, Choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'Model_1'  

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av)

# Command added to avoid any problem with the reported initial loglikelihood. When True, the current iterate is saved after each iteration, in a file named __[modelName].iter
# If such a file exists, the starting values for the estimation are replaced by the values saved in the file.  
biogeme.saveIterations = False 

In [9]:
# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()

# Use print command with above "pandasResults" as argument to get succint results. 
print(pandasResults)

# However, go to directory folder to access detailed results

              Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_Bus    3.877669      0.327207    11.850809  0.000000e+00
ASC_Car    1.013999      0.356375     2.845311  4.436811e-03
Beta_Cost -0.431034      0.070175    -6.142252  8.135967e-10
Beta_TT   -0.344430      0.019443   -17.714589  0.000000e+00
